In [5]:
from pathlib import Path
import sys
import numpy as np
from scipy.constants import g

# Ensure personal Python packages are in path
if (ppfolder := str(Path('..') / 'personal_python_packages')) not in sys.path:
    sys.path.append(ppfolder)
from LNECSPA import LTFdb

def ltf_to_txt(file_path, out_dir):
    """
    Read LNEC .acq file and export to TXT, automatically classifying
    channels as displacement or acceleration based on types or units.
    Displacement units: 'mm' -> convert to meters.
    Acceleration units: 'g'  -> convert to m/s^2.
    After conversion, update units to SI and fill missing types.
    Directly modifies tgtA._data so that write_txt sees the converted arrays.
    """
    file_path = Path(file_path)
    tgtA = LTFdb()
    tgtA.read(file_path)

    # Determine channel indices by type or unit
    acc_inds = []
    disp_inds = []
    for idx, (typ, unit) in enumerate(zip(tgtA.types, tgtA.units)):
        t = (typ or '').strip().lower()
        u = (unit or '').strip().lower()
        if t == 'acceleration' or (not t and u == 'g'):
            acc_inds.append(idx)
            # Fill missing type if empty
            if not tgtA.types[idx].strip():
                tgtA.types[idx] = 'acceleration'
        elif t == 'displacement' or (not t and u == 'mm'):
            disp_inds.append(idx)
            if not tgtA.types[idx].strip():
                tgtA.types[idx] = 'displacement'
        else:
            # Channels with unknown type/unit: skip or handle as needed
            print(f"Channel {idx}: Unknown type '{typ}' and/or unit '{unit}'. Skipping conversion for this channel.")
            continue

    # Convert displacement channels: directly modify tgtA._data
    for ch_idx in disp_inds:
        # Retrieve the 1D data array for this channel
        arr = np.array(tgtA._data[ch_idx], dtype=float)  # ensure float
        original_unit = (tgtA.units[ch_idx] or '').strip().lower()
        if original_unit == 'mm':
            arr = arr * 1e-3  # mm -> m
            tgtA.units[ch_idx] = 'm'
        elif original_unit == 'm':
            # already SI, leave arr as is
            pass
        else:
            print(f"Displacement channel {ch_idx}: Unrecognized unit '{original_unit}'. No conversion applied.")
            # you might choose to leave arr unchanged or handle other units
        # Assign back to internal data list
        tgtA._data[ch_idx] = arr

    # Convert acceleration channels: directly modify tgtA._data
    for ch_idx in acc_inds:
        arr = np.array(tgtA._data[ch_idx], dtype=float)
        original_unit = (tgtA.units[ch_idx] or '').strip().lower()
        if original_unit == 'g':
            arr = arr * g  # g -> m/s^2
            tgtA.units[ch_idx] = 'm/s^2'
        elif original_unit in ['m/s^2', 'm/s²']:
            # already SI
            pass
        else:
            print(f"Acceleration channel {ch_idx}: Unrecognized unit '{original_unit}'. No conversion applied.")
        tgtA._data[ch_idx] = arr

    # Ensure output directory exists
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Determine output filename (append .txt)
    filename = file_path.name + ".txt"
    output_path = out_dir / filename

    # Write the .ltf database to TXT using the updated _data
    tgtA.write_txt(str(output_path))
    print(f"Written converted data to {output_path}")


In [6]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project\LAquilaReducedScale.tgt'
output_folder = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project'
ltf_to_txt(file_path  , output_folder)

Written converted data to C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project\LAquilaReducedScale.tgt.txt


In [9]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\SystemId\pink_noise_40Hz_T3mm.drv'
output_folder = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project'
ltf_to_txt(file_path  , output_folder)

Written converted data to C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project\pink_noise_40Hz_T3mm.drv.txt


In [ ]:
%%script skip
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project\pink_noise_40Hz_T3mm_acq.txt.acq'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\SystemId\pink_noise_40Hz_T3mm_L4mm.drv'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project\LAquilaReducedScale_0.DRV'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\LAquilaReducedScale.tgt'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34.DRV'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34.acq'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\SystemId\pink_noise_40Hz_L4mm.drv'

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\SystemId\pink_noise_40Hz_T3mm.drv'